In [14]:
#Dependencies
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
from datetime import datetime

In [18]:
def metaScrape(page_num, year):
    print(year)
    # request variables
    url_score = f'https://www.metacritic.com/browse/albums/score/metascore/year/filtered?year_selected={year}&distribution=&sort=desc&view=detailed&page={page_num}'
    # bs4 variables
    
    # set user agent for header
    user_agent = {'User-agent': 'Mozilla/5.0'}
    # send response
    response_score = requests.get(url_score, headers = user_agent)
    # scrape website into variable to parse
    soup_score = BeautifulSoup(response_score.text, 'html.parser')
    # create temporary lists for user scores
    userP = []
    userM = []
    userN = []
    # create/initialize dictionary 
    albums_dict = {'artist':[], 'album':[], 'date':[], 'week_num':[], 'meta_score': [], 'user_score':[]}
    # csv variables
    output_path = os.path.join('..','..', 'data', f'albums_{year}.csv')
    
    soup_score.find_all('td', class_='clamp-summary-wrap')
    # create soup 
    for artist in soup_score.find_all('td', class_='clamp-summary-wrap'):
        # scrape album name
        albums_dict['album'].append(artist.find('a', class_= 'title').text)
        # scrape artist name and strip white space and extra characters
        albums_dict['artist'].append(artist.find('div', class_='artist').text.strip().lstrip('by '))
        # scrape date
        albums_dict['date'].append(artist.find('div', class_='clamp-details').find('span').text)
        # scrape meta_score, handle for changes in class name, convert data type of score to int and append to dict
        # except set to pass since all alubms have a score

        try:
            albums_dict['meta_score'].append(int(artist.find('div', class_='metascore_w large release positive').text))  
        except:
            pass
        try:
            albums_dict['meta_score'].append(int(artist.find('div', class_='metascore_w large release mixed').text))  
        except:
            pass 
        try:
            albums_dict['meta_score'].append(int(artist.find('div', class_='metascore_w large release negative').text))  
        except:
            pass
        # scrape user score, handle errors for tbd/class name and append to temp list
        try:
            userP.append(float(artist.find('div', class_='metascore_w user large release positive').text))  
        except:
            userP.append(0)
        try:
            userM.append(float(artist.find('div', class_='metascore_w user large release mixed').text))  
        except:
            userM.append(0)
        try:
            userN.append(float(artist.find('div', class_='metascore_w user large release negative').text))  
        except:
            userN.append(0)
    # merge user score by filtering scores from tbd using data type in temporary lists, convert data type of scores to int and append to dictionary        
    for a, b, c in zip(userP, userM, userN):
        if isinstance(a, float):
            albums_dict['user_score'].append(int(a * 10))
        elif isinstance(b, float):
            albums_dict['user_score'].append(int(b * 10))
        elif isinstance(c, float):
            albums_dict['user_score'].append(int(c * 10))
        else:
            albums_dict['user_score'].append(c)
#     create week_num key and values for weekly playlists
    for dates in albums_dict['date']:
        albums_dict['week_num'].append((datetime.strptime(dates, '%B %d, %Y')).isocalendar()[1])

    # write dictionary to csv
    # create header
    fields = ['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score'] 
    # create variable for data to be written
    data = zip(albums_dict['artist'], albums_dict['album'], albums_dict['date'], albums_dict['week_num'], albums_dict['meta_score'], albums_dict['user_score'])
    # create file and write csv
    if page_num == 0 or False:
        with open(output_path, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(fields)
            for d in data:
                writer.writerow(d)
    else:
        with open(output_path, 'a') as csvfile:
            writer = csv.writer(csvfile)
            for d in data:
                writer.writerow(d)

In [19]:
def metaScorePages():
    # find number of pages for albums 2020
    url_pages = 'https://www.metacritic.com/browse/albums/score/metascore/year/filtered?year_selected=2020&distribution=&sort=desc&view=detailed&page=0'
    # set user agent for header
    user_agent = {'User-agent': 'Mozilla/5.0'}
    # send response
    response_pages = requests.get(url_pages, headers = user_agent)
    # scrape website into variable to parse
    soup_pages = BeautifulSoup(response_pages.text, 'html.parser')
    # print(soup_pages)
    # create temporary lists for user scores

    pages = int(soup_pages.find("li", class_="page last_page").next_element.text)
    for year in range(2000, 2021):
        for page in range(pages):
            metaScrape(page, year)

In [20]:
metaScorePages()

2000
2000
2000
2000
2000
2001
2001
2001
2001
2001
2002
2002
2002
2002
2002
2003
2003
2003
2003
2003
2004
2004
2004
2004
2004
2005
2005
2005
2005
2005
2006
2006
2006
2006
2006
2007
2007
2007
2007
2007
2008
2008
2008
2008
2008
2009
2009
2009
2009
2009
2010
2010
2010
2010
2010
2011
2011
2011
2011
2011
2012
2012
2012
2012
2012
2013
2013
2013
2013
2013
2014
2014
2014
2014
2014
2015
2015
2015
2015
2015
2016
2016
2016
2016
2016
2017
2017
2017
2017
2017
2018
2018
2018
2018
2018
2019
2019
2019
2019
2019
2020
2020
2020
2020
2020
